## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,73.54,89,72,5.99,broken clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,74,23,12.66,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,80.98,81,91,1.57,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,58.26,93,81,3.51,broken clouds
4,4,Dingle,PH,10.9995,122.6711,86.92,70,75,6.04,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,80.65,74,23,12.66,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,80.98,81,91,1.57,overcast clouds
4,4,Dingle,PH,10.9995,122.6711,86.92,70,75,6.04,light rain
5,5,Rapid Valley,US,44.0625,-103.1463,82.60,37,1,6.91,clear sky
7,7,Beruwala,LK,6.4788,79.9828,80.29,85,100,11.18,overcast clouds
8,8,Malkapur,IN,20.8833,76.2000,82.99,63,99,3.69,overcast clouds
9,9,Yenagoa,NG,4.9247,6.2642,76.28,94,88,3.33,overcast clouds
10,10,Kloulklubed,PW,7.0419,134.2556,84.70,71,17,1.50,few clouds
23,23,Rantauprapat,ID,2.1000,99.8333,84.02,72,100,2.15,overcast clouds
29,29,Ixtapa,MX,20.7000,-105.2000,82.22,83,90,6.91,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.2078,-159.7750,80.65,74,23,12.66,few clouds
2,2,Bengkulu,ID,-3.8004,102.2655,80.98,81,91,1.57,overcast clouds
4,4,Dingle,PH,10.9995,122.6711,86.92,70,75,6.04,light rain
5,5,Rapid Valley,US,44.0625,-103.1463,82.60,37,1,6.91,clear sky
7,7,Beruwala,LK,6.4788,79.9828,80.29,85,100,11.18,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
2,Bengkulu,ID,80.98,overcast clouds,-3.8004,102.2655,
4,Dingle,PH,86.92,light rain,10.9995,122.6711,
5,Rapid Valley,US,82.60,clear sky,44.0625,-103.1463,
7,Beruwala,LK,80.29,overcast clouds,6.4788,79.9828,
8,Malkapur,IN,82.99,overcast clouds,20.8833,76.2000,
9,Yenagoa,NG,76.28,overcast clouds,4.9247,6.2642,
10,Kloulklubed,PW,84.70,few clouds,7.0419,134.2556,
23,Rantauprapat,ID,84.02,overcast clouds,2.1000,99.8333,
29,Ixtapa,MX,82.22,overcast clouds,20.7000,-105.2000,


In [8]:
import numpy as np

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
2,Bengkulu,ID,80.98,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
4,Dingle,PH,86.92,light rain,10.9995,122.6711,SEGAYA PROPERTY
5,Rapid Valley,US,82.60,clear sky,44.0625,-103.1463,Fairfield by Marriott Rapid City
7,Beruwala,LK,80.29,overcast clouds,6.4788,79.9828,The Palms Hotel
8,Malkapur,IN,82.99,overcast clouds,20.8833,76.2000,Hotel Sarthi International
9,Yenagoa,NG,76.28,overcast clouds,4.9247,6.2642,Aridolf Resort Wellness and Spa
10,Kloulklubed,PW,84.70,few clouds,7.0419,134.2556,Storyboard Beach Resort
23,Rantauprapat,ID,84.02,overcast clouds,2.1000,99.8333,Permata Land Hotel & Resort
29,Ixtapa,MX,82.22,overcast clouds,20.7000,-105.2000,Hotel Plaza Providencia


In [11]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))